# <u><b><h1>Web Scraping<h1></b></u>

# <u>Creating a dataframe with datas from IMDb top 250 movies<u>

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

url de notre page

In [2]:

url = "https://www.imdb.com/list/ls098063263/?sort=moviemeter,asc&st_dt=&mode=detail&page=1"

In [3]:
def all_page_link(start_url):
    all_urls = []
    url = start_url
    while(url != None):           
        all_urls.append(url)
        soup = BeautifulSoup(requests.get(url).text,"html.parser")
        next_links = soup.find_all(class_='flat-button lister-page-next next-page')   
        if (len(next_links) == 0):        
            url = None
        else:
            next_page = "https://www.imdb.com" + next_links[0].get('href')
            url = next_page
    return all_urls

In [4]:
def director_and_actor(Director_and_star):

    Director_and_star =  Director_and_star.replace("\n","")
    Director_and_star = Director_and_star.replace("|","")
    Director_and_star = Director_and_star.split("Stars:")
    Director_and_star[0] = Director_and_star[0].replace("Director:","")
    Director_and_star[0] = Director_and_star[0].replace("Directors:","")

    for i in range(10):
        Director_and_star[0] = Director_and_star[0].replace("  "," ")
    director = Director_and_star[0][1:]
    stars = Director_and_star[1]
    stars = stars.replace(":", "")

    return director, stars


In [5]:
def votes_and_gross_converter(votes_and_gross):
    votes_and_gross_list = []

    # Parcours chaque élément de la liste d'objets BeautifulSoup
    for i in votes_and_gross:
        # Ajoute le texte de l'élément à la liste
        votes_and_gross_list.append(i.text)

    # Vérifie la longueur de la liste
    if len(votes_and_gross) == 2:
        # Si la liste contient deux éléments, le premier est le nombre de votes, et le deuxième est le montant brut
        votes = votes_and_gross_list[0]
        gross = votes_and_gross_list[1]
    else:
        # Si la liste ne contient qu'un élément, il s'agit du nombre de votes, et le montant brut est défini sur None
        votes = votes_and_gross_list[0]
        gross = None
    
    # Retourne le nombre de votes et le montant brut
    return votes, gross



In [7]:
main_array = []
for url in tqdm(all_page_link("https://www.imdb.com/list/ls098063263/?sort=moviemeter,asc&st_dt=&mode=detail&page=1")):     #Runs the function for all the pages.
    soup = BeautifulSoup(requests.get(url).text,"html.parser")         
    for link in soup.find_all(class_='lister-item-content'):
        id_text = link.find('span', {"class": "lister-item-index unbold text-primary"}).text[:-1]
        id = int(id_text.replace(",", ""))

        name = link.find('a').text
        year = link.find('span',{"class":"lister-item-year text-muted unbold"}).text[1:5]
        run_time = link.find('span',{"class":"runtime"}).text
        genre = link.find('span',{"class":"genre"}).text[1:]
        rating = link.find('span',{"class":"ipl-rating-star__rating"}).text
        about = link.find_all('p')[1].text[5:]
        director,actors = director_and_actor(link.find_all('p',{"class":"text-muted text-small"})[1].text)
        votes, gross = votes_and_gross_conveter(link.find_all('span',{"name":"nv"}))
        votes = int(votes.replace(",",""))
        list_of_all = [id,name,year,run_time,genre,rating,about,director,actors,votes,gross]
        main_array.append(list_of_all)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:29<00:00,  9.00s/it]


In [8]:

index = ["id","name","year","run_time","genre","rating","about","director","actors","votes","gross"]

In [9]:
df = pd.DataFrame(main_array,columns=index)  

In [10]:
df.head(5)

,id,name,year,run_time,genre,rating,about,director,actors,votes,gross
0,1,Oppenheimer,I) (,180 min,"Biography, Drama, History",8.4,"story of American scientist, J. Robert Oppenhe...",Christopher Nolan,"Cillian Murphy, Emily Blunt, Matt Damon, Rober...",547233,None
1,2,Indiana Jones and the Dial of Destiny,2023,154 min,"Action, Adventure, Sci-Fi",6.6,aeologist Indiana Jones races against time to ...,James Mangold,"Harrison Ford, Phoebe Waller-Bridge, Antonio B...",152583,None
2,3,Home Alone,1990,103 min,"Comedy, Family",7.7,"ight-year-old troublemaker, mistakenly left ho...",Chris Columbus,"Macaulay Culkin, Joe Pesci, Daniel Stern, John...",626006,$285.76M
3,4,Love Actually,2003,135 min,"Comedy, Drama, Romance",7.6,ows the lives of eight very different couples ...,Richard Curtis,"Hugh Grant, Martine McCutcheon, Liam Neeson, L...",514724,$59.70M
4,5,Elf,2003,97 min,"Adventure, Comedy, Family",7.1,"ed as an oversized elf, Buddy travels from the...",Jon Favreau,"Will Ferrell, James Caan, Bob Newhart, Zooey D...",298236,$173.40M


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1000 non-null   int64 
 1   name      1000 non-null   object
 2   year      1000 non-null   object
 3   run_time  1000 non-null   object
 4   genre     1000 non-null   object
 5   rating    1000 non-null   object
 6   about     1000 non-null   object
 7   director  1000 non-null   object
 8   actors    1000 non-null   object
 9   votes     1000 non-null   int64 
 10  gross     956 non-null    object
dtypes: int64(2), object(9)
memory usage: 86.1+ KB


## <b><u> fonction pour faire appel a la fonction pour enregistrer dans csv 

In [12]:
def DataFrame_to_csv():   
    name = input("Enter le nom de ficheir que voulais stockes sinon va enregistrer sous nom \"top_1000_IMDb_movies.csv\" ")
    if(name==""):
        print("votre nom de fichier est :-","top_1000_IMDb_movies.csv")
        df.to_csv("top_1000_IMDb_movies.csv")
    elif(name[-4:] == ".csv"):
        print(" votre nom de fichier est:-",name)
        df.to_csv(name)
    else:
        print("votre nom de fichier est :-",(name+".csv"))
        df.to_csv(name+".csv")

In [14]:
DataFrame_to_csv()

Enter the name that you wantto give to the csv file. If no input is given the csv file will take a name as "top_1000_IMDb_movies.csv"  


Your File Name is :- top_1000_IMDb_movies.csv


In [15]:
DataFrame_to_csv()

Enter the name that you wantto give to the csv file. If no input is given the csv file will take a name as "top_1000_IMDb_movies.csv"  


Your File Name is :- top_1000_IMDb_movies.csv


In [18]:
DataFrame_to_csv()

Enter the name that you wantto give to the csv file. If no input is given the csv file will take a name as "top_1000_IMDb_movies.csv"  


Your File Name is :- top_1000_IMDb_movies.csv
